# Hock-Schittkowsky 71 - Degenerate
(Adapted from Problem number [71](https://www.coin-or.org/Ipopt/documentation/node20.html) from the Hock-Schittkowsky test suite and augmented with the formulation 4.2 of Curtis, et al. SIAM Journal on Optimization. 2009 Sep 16;20(3):1224-49.)

> **Requirements**
> - python3.X
> - pyomo 5.X.X
> - [ipopt](https://github.com/coin-or/Ipopt) 3.12.12 (tested with)

### 0. Introduction

In this problem, an ill-posed problem is tested with *ipopt*; to demonstrate the effects of "bad" modelling.
Firstly, consider the problem:

\begin{align}
\text{minimize} \quad & x_{1}x_{4} \left(x_{1} + x_{2} + x_{3} \right) + x_{3} \\
\text{subject to} \quad & x_{1}x_{2}x_{3}x_{4} \geq 25, \\
& x_{1}^{2} + x_{2}^{2} + x_{3}^{2} + x_{4}^{2} = 40 \\
& 1 \leq x_{1}, x_{2}, x_{3}, x_{4} \leq 5,
\end{align}

In the following figure the feasible region is displayed: inside the elipse (first constraint) and below the dotted line (second constraint). Furthermore, the contours of the *barrier* objective function are shown.

![Forsgren](./img/forsgren1.png)  
Figure: The feasible region and objective function level-curves (Credit: Forsgren et al 2002)

Clearly, if *ipopt* is used as a solver, the initial guess will be essential when solving the problem.

### 1. Package imports and model declaration

As usual the pyomo enviroment has to be imported first.

In [138]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
m = ConcreteModel()  #: concrete model

### 2. Variables

In this problem there are two variables. A `Set` object is not necesary, though it could be used to create an indexed variable `x`.  
Since, this problem is nonlinear, it is *necesary* to specify an initial guess. In pyomo, this is usually done at construction level using the `initialize` keyword from the constructor.  
For a simple variable (not indexed) a floating point number is sufficient.

In [139]:
m.i = Set(initialize=[1,2,3,4])

In [140]:
x_guess = {1: 1, 2: 5, 3: 5, 4:1}

In [141]:
m.x = Var(m.i, initialize=x_guess, bounds=(1,5))

However, if the variable is not initized at the construction, its initial value often set to $\max (0, x_{lb})$. This is problematic in several situations as the local linearizations of the constraints might not define a well-posed problem. 

### 3. Constraints and Objective

Singleton constraints can have their respective expressions declared directly at construction. This is done with the `expr` keyword.

In [142]:
#m.c1 = Constraint(expr=-m.x1 ** 2 + m.x2 >= 0)
#m.c2 = Constraint(expr=3 * m.x1  ** 2 - m.x2 >= 0)

In [143]:
#m.c1 = Constraint(expr=m.x1 >= 0)
#m.c2 = Constraint(expr=m.x2 >= 0)
#m.c3 = Constraint(expr=4 - m.x1 ** 2 - 4 * m.x2 ** 2 >= 0)
#m.c4 = Constraint(expr=5 - (m.x1 - 2) ** 2 - m.x2 ** 2 >= 0)

In [144]:
#.c1 = Constraint(expr=m.x1 * m.x2 ==0)
#.c2 = Constraint(expr=m.x2** 2 >= 1)

In [145]:
m.e1 = Expression(initialize=m.x[1]**2 + m.x[2]**2 + m.x[3]**2 + m.x[4]**2 - 40)

In [146]:
m.c0 = Constraint(expr=m.x[1] * m.x[2] * m.x[3] * m.x[4] >= 25)
m.c1 = Constraint(expr=m.e1 == 0)
m.c2 = Constraint(expr=m.e1 - m.e1 ** 2 == 0)

Similarly, the objective function is constructed.

In [147]:
m.of = Objective(expr=m.x[1] * m.x[1] * (m.x[1] + m.x[2] + m.x[3]) + m.x[3], sense=minimize)

### 4. Solution and discussion

To solve this problem, *ipopt* is used. *ipopt* is an interior-point optimization algorithm. It solves a sequence of problems called *barrier* subproblems, in which the variable bounds are moved into the objective function.  
In pyomo, it is possible to turn off the solver ouput using the keyword `tee` of the `solve()` method. Also, note that the status of the solver is returned as a result of the solver call.

In [148]:
opt = SolverFactory('ipopt')
res = opt.solve(m, tee=True)
print(res['Solver'])   #: get the status after solver call

Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        8
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Tot

ValueError: Cannot load a SolverResults object with bad status: error

One way to show the results is by means of `.display()` method for variables.  

In [ ]:
m.x.display()

*ipopt* found a *local* solution. From the Figure at the introductuction section, it is clear that if the initial guess is perturbed; the results might change.  
In order to test this issue, one can set the initial guess to a different value. For a singleton variable this is done with the `set_value()` method.  
`set_value()` will have different behaviour if the variable is indexed.

In [ ]:
m.x1.set_value(20)  #: Change the initial guesses            
m.x2.set_value(20)

Then the model can be resolved:

In [ ]:
with open("ipopt.opt", "w") as f:
    f.write("print_info_string yes\n")

In [ ]:
res = opt.solve(m, tee=True)
print(res['Solver'])

And the results displayed in similar way.

In [ ]:
m.x1.display()
m.x2.display()

Note that the results have changed. This illustrates the issue of providing *good* initial guesses for nonlinear problems.  
The real solution of this problem is in fact $x^{*} = (\dfrac{3}{4}, \dfrac{-1}{2})$.